# Startegi : Using each label using neural network(regression)
## each label x-> x

In [1]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [2]:
path='data/train.csv'

In [3]:
df=pd.read_csv(path, index_col=0)
df=df[['x','y','z','x_sim','y_sim','z_sim', 'sat_id']]
# df=df.drop('epoch', axis=1)
df.head()

,x,y,z,x_sim,y_sim,z_sim,sat_id
id,,,,,,,
0,-8855.823863,13117.780146,-20728.353233,-8843.131454,13138.221690,-20741.615306,0
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556,0
2,-10578.684043,-10180.467460,-24238.280949,-10571.858472,-10145.939908,-24271.169776,0
3,-9148.251857,-20651.437460,-20720.381279,-9149.620794,-20618.200201,-20765.019094,0
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986,0


In [4]:
mydata=df.copy()

In [5]:
mydata.isna().sum()

x         0
y         0
z         0
x_sim     0
y_sim     0
z_sim     0
sat_id    0
dtype: int64

In [6]:
mydata = mydata.dropna()
mydata.shape

(649912, 7)

In [7]:
train_mydata = mydata.sample(frac=0.8,random_state=0)
test_mydata = mydata.drop(train_mydata.index)
test_mydata.head()

,x,y,z,x_sim,y_sim,z_sim,sat_id
id,,,,,,,
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556,0
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986,0
15,17281.246143,-2011.121378,39970.739873,17241.395285,-2016.770194,39958.759155,0
22,-1447.724638,28032.228987,-3918.192838,-1461.093061,28026.085525,-3959.767549,0
26,-10583.521489,-10145.003880,-24243.775722,-10574.016380,-10147.909472,-24270.149411,0


In [8]:
#a=train_mydata.shape
#b=test_mydata.shape
#print(a,b)
print(train_mydata.shape, test_mydata.shape)
train_mydata.head()

(519930, 7) (129982, 7)


,x,y,z,x_sim,y_sim,z_sim,sat_id
id,,,,,,,
162906,-22445.306373,5848.139687,805.906588,-21786.206588,7375.036638,391.178832,81
123662,-9086.781838,7654.307240,-6721.299832,-9545.479053,7482.025507,-6387.708079,60
901720,-33475.100342,-37591.455624,-3269.507831,-33263.131053,-37783.198553,-3617.165346,449
308669,11617.034853,2343.826977,-1821.565305,10718.558812,5174.769686,-1069.308627,166
1154759,25113.931453,-473.679010,41867.498593,19229.123354,12147.342145,31835.248429,560


In [9]:
label=['Vx','Vy','Vz','x','y','z'][3:]
train_stats = train_mydata.describe()
for i in label:
  train_stats.pop(i)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
x_sim,519930.0,-941.331006,25387.037253,-255037.952966,-13139.771199,-612.877803,11194.372299,200961.158000
y_sim,519930.0,319.853163,25603.245579,-281784.430040,-11758.892200,501.609913,12558.620911,209619.553857
z_sim,519930.0,-77.075471,20670.833347,-180250.922287,-7963.461284,57.093264,8253.028193,198526.091552
sat_id,519930.0,308.718101,171.427627,0.000000,166.000000,312.000000,461.000000,599.000000


In [10]:
train_labels =train_mydata[label]
test_labels = test_mydata[label]

In [11]:
print(train_labels.head(), test_labels.head())

                    x             y             z
id                                               
162906  -22445.306373   5848.139687    805.906588
123662   -9086.781838   7654.307240  -6721.299832
901720  -33475.100342 -37591.455624  -3269.507831
308669   11617.034853   2343.826977  -1821.565305
1154759  25113.931453   -473.679010  41867.498593                x             y             z
id                                          
1  -10567.672384   1619.746066 -24451.813271
4   -6719.092336 -28929.061629 -14938.907967
15  17281.246143  -2011.121378  39970.739873
22  -1447.724638  28032.228987  -3918.192838
26 -10583.521489 -10145.003880 -24243.775722


In [12]:
def norm(x):
  try:
    return (x - train_stats['mean']) / train_stats['std']
  except:
    print(x)

In [13]:
normed_train_data = norm(train_mydata.drop(label, axis=1))
normed_test_data = norm(test_mydata.drop(label, axis=1))

In [14]:
a=normed_test_data.describe()
a.transpose()

,count,mean,std,min,25%,50%,75%,max
x_sim,129982.0,-0.001650,0.997447,-10.002493,-0.483600,0.011332,0.477304,7.940276
y_sim,129982.0,0.000919,1.000829,-11.059974,-0.467484,0.011119,0.476295,8.154777
z_sim,129982.0,-0.000651,0.999315,-8.628536,-0.380103,0.006436,0.399140,9.581585
sat_id,129982.0,0.000357,1.001616,-1.800866,-0.832527,0.019145,0.888316,1.693320


In [15]:
normed_test_data.shape

(129982, 4)

In [16]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = 2, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 165,249
Trainable params: 165,249
Non-trainable params: 0
_________________________________________________________________


In [17]:
train_labels.head()

,x,y,z
id,,,
162906,-22445.306373,5848.139687,805.906588
123662,-9086.781838,7654.307240,-6721.299832
901720,-33475.100342,-37591.455624,-3269.507831
308669,11617.034853,2343.826977,-1821.565305
1154759,25113.931453,-473.679010,41867.498593


In [19]:
for i in label[0]:
    NN_model.fit(normed_train_data[['sat_id',i+'_sim']], train_labels[i], validation_data=(normed_test_data[['sat_id',i+'_sim']], test_labels[i]), epochs=10)
    test_predictions = NN_model.predict(normed_test_data[['sat_id',i+'_sim']])
    test_mydata[i+'_pred']=test_predictions

Train on 519930 samples, validate on 129982 samples
Epoch 1/10
519930/519930 [==============================] - 38s 72us/step - loss: 3142.0706 - mean_absolute_error: 3142.0740 - val_loss: 3125.9136 - val_mean_absolute_error: 3125.9080
Epoch 2/10
519930/519930 [==============================] - 38s 73us/step - loss: 3137.9510 - mean_absolute_error: 3137.9512 - val_loss: 3169.1665 - val_mean_absolute_error: 3169.1636
Epoch 3/10
519930/519930 [==============================] - 39s 75us/step - loss: 3137.1691 - mean_absolute_error: 3137.1775 - val_loss: 3197.9107 - val_mean_absolute_error: 3197.9138
Epoch 4/10
519930/519930 [==============================] - 38s 74us/step - loss: 3136.1201 - mean_absolute_error: 3136.1218 - val_loss: 3119.5875 - val_mean_absolute_error: 3119.5889
Epoch 5/10
519930/519930 [==============================] - 38s 73us/step - loss: 3134.6685 - mean_absolute_error: 3134.6663 - val_loss: 3123.4484 - val_mean_absolute_error: 3123.4468
Epoch 6/10
519930/519930 [==

In [20]:
test_mydata.head()

,x,y,z,x_sim,y_sim,z_sim,sat_id,y_pred,z_pred,x_pred
id,,,,,,,,,,
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556,0,1699.232178,-24427.830078,-10729.541992
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986,0,-28538.695312,-14938.091797,-6854.264648
15,17281.246143,-2011.121378,39970.739873,17241.395285,-2016.770194,39958.759155,0,-2046.540405,39941.507812,17365.035156
22,-1447.724638,28032.228987,-3918.192838,-1461.093061,28026.085525,-3959.767549,0,27811.888672,-3870.237061,-1399.343872
26,-10583.521489,-10145.003880,-24243.775722,-10574.016380,-10147.909472,-24270.149411,0,-9931.518555,-24224.523438,-10748.294922


In [21]:
def smape(satellite_predicted_values, satellite_true_values):
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/(np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [22]:
for i in ['_sim', '_pred']:
    real=test_mydata[label].values
    comp=test_mydata[[j+i for j in label]].values
    print('smape real vs %s : %f'%(i, smape(real, comp)))

smape real vs _sim : 0.184052
smape real vs _pred : 0.185913
